In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from joblib import dump, load
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from itertools import cycle
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import sklearn.metrics as metrics

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


In [2]:
import pandas as pd
import numpy as np
import sys
import keras
import sklearn
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
from tensorflow.keras.utils import to_categorical
import sklearn.preprocessing
from sklearn import metrics
from scipy.stats import zscore
from tensorflow.keras.utils import get_file, plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
print(pd.__version__)
print(np.__version__)
print(sys.version)
print(sklearn.__version__)


1.4.1
1.21.2
3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]
1.0.2


In [3]:
#Loading training set into dataframe
df = pd.read_csv('./../dataset/NSL-KDD/KDDTrain+_20Percent.txt', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [4]:
#Loading testing set into dataframe
qp = pd.read_csv('./../dataset/NSL-KDD/KDDTest-21.txt', header=None)
qp.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,13,tcp,telnet,SF,118,2425,0,0,0,0,...,0.38,0.12,0.04,0.0,0.00,0.00,0.12,0.3,guess_passwd,2
1,0,udp,private,SF,44,0,0,0,0,0,...,1.00,0.01,0.01,0.0,0.00,0.00,0.00,0.0,snmpguess,12
2,0,tcp,telnet,S3,0,44,0,0,0,0,...,0.31,0.61,0.00,0.0,0.21,0.68,0.60,0.0,processtable,18
3,0,udp,private,SF,53,55,0,0,0,0,...,1.00,0.00,0.87,0.0,0.00,0.00,0.00,0.0,normal,17
4,0,tcp,private,SH,0,0,0,0,0,0,...,0.06,1.00,1.00,0.0,1.00,1.00,0.00,0.0,nmap,17


In [5]:
#Reset column names for training set
df.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level']
df.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,subclass,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [6]:
#Reset column names for testing set
qp.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
'num_access_files', 'num_outbound_cmds', 'is_host_login',
'is_guest_login', 'count', 'srv_count', 'serror_rate',
'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
'dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level']
qp.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,subclass,difficulty_level
0,13,tcp,telnet,SF,118,2425,0,0,0,0,...,0.38,0.12,0.04,0.0,0.00,0.00,0.12,0.3,guess_passwd,2
1,0,udp,private,SF,44,0,0,0,0,0,...,1.00,0.01,0.01,0.0,0.00,0.00,0.00,0.0,snmpguess,12
2,0,tcp,telnet,S3,0,44,0,0,0,0,...,0.31,0.61,0.00,0.0,0.21,0.68,0.60,0.0,processtable,18
3,0,udp,private,SF,53,55,0,0,0,0,...,1.00,0.00,0.87,0.0,0.00,0.00,0.00,0.0,normal,17
4,0,tcp,private,SH,0,0,0,0,0,0,...,0.06,1.00,1.00,0.0,1.00,1.00,0.00,0.0,nmap,17


In [7]:
#accessing names of training columns
lst_names = df.columns # returns a list of column names
lst_names


Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level'],
      dtype='object')

In [8]:
#accessing names of testing columns
testlst_names = qp.columns
testlst_names

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'subclass', 'difficulty_level'],
      dtype='object')

In [9]:
#Dropping the last columns of training set
df = df.drop('difficulty_level', 1) # we don't need it in this project
df.shape

/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/129585363.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('difficulty_level', 1) # we don't need it in this project


(25192, 42)

In [10]:
#Dropping the last columns of testing set
qp = qp.drop('difficulty_level', 1)
qp.shape

/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/700357697.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  qp = qp.drop('difficulty_level', 1)


(11850, 42)

In [11]:
df.isnull().values.any()

False

In [12]:
qp.isnull().values.any()

False

In [13]:
#defining col list
cols = ['protocol_type','service','flag']
cols

['protocol_type', 'service', 'flag']

In [14]:
#One-hot encoding
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each, 1)
    return df


In [15]:
#Merging train and test data
combined_data = pd.concat([df,qp])

In [16]:
#Applying one hot encoding to combined data
combined_data = one_hot(combined_data,cols)

/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/2521078246.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/2521078246.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/2521078246.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)


In [17]:
#Function to min-max normalize
def normalize(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with normalized specified features
    """
    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [18]:
#Dropping subclass column for training set
tmp = combined_data.pop('subclass')

In [19]:
#Normalizing training set
new_train_df = normalize(combined_data,combined_data.columns)

In [20]:
#Fixing labels for training set
classlist = []
check1 = ("apache2","back","land","neptune","mailbomb","pod","processtable","smurf","teardrop","udpstorm","worm")
check2 = ("ipsweep","mscan","nmap","portsweep","saint","satan")
check3 = ("buffer_overflow","loadmodule","perl","ps","rootkit","sqlattack","xterm")
check4 = ("ftp_write","guess_passwd","httptunnel","imap","multihop","named","phf","sendmail","Snmpgetattack","spy","snmpguess","warezclient","warezmaster","xlock","xsnoop")

DoSCount=0
ProbeCount=0
U2RCount=0
R2LCount=0
NormalCount=0

for item in tmp:
    if item in check1:
        classlist.append("DoS")
        DoSCount=DoSCount+1
    elif item in check2:
        classlist.append("Probe")
        ProbeCount=ProbeCount+1
    elif item in check3:
        classlist.append("U2R")
        U2RCount=U2RCount+1
    elif item in check4:
        classlist.append("R2L")
        R2LCount=R2LCount+1
    else:
        classlist.append("Normal")
        NormalCount=NormalCount+1   

In [21]:
DoSCount

13578

In [22]:
#Appending class column to training set
new_train_df["Class"] = classlist


/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/1289121422.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_train_df["Class"] = classlist


In [23]:
new_train_df["Class"].value_counts()

Normal    15779
DoS       13578
Probe      4691
R2L        2916
U2R          78
Name: Class, dtype: int64

In [24]:
new_train_df.shape

(37042, 120)

In [25]:
new_train_df.isnull().values.any()

False

In [26]:
y_train=new_train_df["Class"]
y_train

0        Normal
1        Normal
2           DoS
3        Normal
4        Normal
          ...  
11845    Normal
11846    Normal
11847     Probe
11848     Probe
11849       R2L
Name: Class, Length: 37042, dtype: object

In [27]:
y_train.isnull().values.any()

False

In [29]:
import os
combined_data_X = new_train_df.drop('Class', 1)
combined_data_X

combined_data_X.to_csv(os.path.join('', 'prepocessedTrain_new122dim.csv'),index=False)

/var/folders/82/wc2_x5cd7s79_djrrs742b1w0000gn/T/ipykernel_86969/1344658330.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  combined_data_X = new_train_df.drop('Class', 1)


In [30]:
oos_pred = []

In [31]:
from sklearn.model_selection import StratifiedKFold

In [32]:
kfold = StratifiedKFold(n_splits=6,shuffle=True,random_state=42)
kfold.get_n_splits(combined_data_X,y_train)

6

In [37]:
#Bidirectional RNN
batch_size = 32
model = Sequential()
model.add(Convolution1D(64, kernel_size=119, 
                            padding="same",
                            activation="relu",input_shape=(119, 1)))
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(64, return_sequences=False))) 
model.add(Reshape((128, 1), input_shape = (128, )))
    
model.add(MaxPooling1D(pool_size=(5)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=False))) 
    
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [38]:
for layer in model.layers:
    print(layer.output_shape)

(None, 119, 64)
(None, 23, 64)
(None, 23, 64)
(None, 128)
(None, 128, 1)
(None, 25, 1)
(None, 25, 1)
(None, 256)
(None, 256)
(None, 5)
(None, 5)


In [39]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 119, 64)           7680      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 23, 64)           0         
 1D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 23, 64)           256       
 hNormalization)                                                 
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 reshape_1 (Reshape)         (None, 128, 1)            0         
                                                      

In [40]:
for train_index, test_index in kfold.split(combined_data_X,y_train):
    train_X, test_X = combined_data_X.iloc[train_index], combined_data_X.iloc[test_index]
    train_y, test_y = y_train.iloc[train_index], y_train.iloc[test_index]
    
    print("train index:",train_index)
    print("test index:",test_index)
    
    x_columns_train = new_train_df.columns.drop('Class')
    x_train_array = train_X[x_columns_train].values
    x_train_1=np.reshape(x_train_array, (x_train_array.shape[0], x_train_array.shape[1], 1))
    
    dummies = pd.get_dummies(train_y) # Classification
    outcomes = dummies.columns
    num_classes = len(outcomes)
    y_train_1 = dummies.values
    
    x_columns_test = new_train_df.columns.drop('Class')
    x_test_array = test_X[x_columns_test].values
    x_test_2=np.reshape(x_test_array, (x_test_array.shape[0], x_test_array.shape[1], 1))
    
    dummies_test = pd.get_dummies(test_y) # Classification
    outcomes_test = dummies_test.columns
    num_classes = len(outcomes_test)
    y_test_2 = dummies_test.values
    
    model.fit(x_train_1, y_train_1,validation_data=(x_test_2,y_test_2), epochs=10)
    
    pred = model.predict(x_test_2)
    pred = np.argmax(pred,axis=1)
    y_eval = np.argmax(y_test_2,axis=1)
    score = metrics.accuracy_score(y_eval, pred)
    oos_pred.append(score)
    print("Validation score: {}".format(score))

train index: [    0     1     2 ... 37039 37040 37041]
test index: [    7    13    21 ... 37035 37036 37037]
Epoch 1/10
965/965 [==============================] - 63s 61ms/step - loss: 0.2371 - accuracy: 0.9217 - val_loss: 0.2085 - val_accuracy: 0.9245
Epoch 2/10
965/965 [==============================] - 70s 73ms/step - loss: 0.1471 - accuracy: 0.9489 - val_loss: 0.1334 - val_accuracy: 0.9556
Epoch 3/10
965/965 [==============================] - 72s 75ms/step - loss: 0.1227 - accuracy: 0.9548 - val_loss: 0.1291 - val_accuracy: 0.9538
Epoch 4/10
965/965 [==============================] - 75s 77ms/step - loss: 0.1065 - accuracy: 0.9603 - val_loss: 0.1049 - val_accuracy: 0.9661
Epoch 5/10
965/965 [==============================] - 75s 77ms/step - loss: 0.0983 - accuracy: 0.9626 - val_loss: 0.1022 - val_accuracy: 0.9611
Epoch 6/10
965/965 [==============================] - 66s 68ms/step - loss: 0.0909 - accuracy: 0.9652 - val_loss: 0.0993 - val_accuracy: 0.9640
Epoch 7/10
965/965 [=======

965/965 [==============================] - 68s 70ms/step - loss: 0.0394 - accuracy: 0.9828 - val_loss: 0.0408 - val_accuracy: 0.9849
Epoch 3/10
965/965 [==============================] - 70s 72ms/step - loss: 0.0373 - accuracy: 0.9841 - val_loss: 0.0469 - val_accuracy: 0.9843
Epoch 4/10
965/965 [==============================] - 96s 99ms/step - loss: 0.0390 - accuracy: 0.9838 - val_loss: 0.0462 - val_accuracy: 0.9804
Epoch 5/10
965/965 [==============================] - 128s 132ms/step - loss: 0.0376 - accuracy: 0.9839 - val_loss: 0.0424 - val_accuracy: 0.9840
Epoch 6/10
965/965 [==============================] - 108s 112ms/step - loss: 0.0367 - accuracy: 0.9842 - val_loss: 0.0455 - val_accuracy: 0.9823
Epoch 7/10
965/965 [==============================] - 80s 83ms/step - loss: 0.0358 - accuracy: 0.9842 - val_loss: 0.0517 - val_accuracy: 0.9799
Epoch 8/10
965/965 [==============================] - 86s 89ms/step - loss: 0.0376 - accuracy: 0.9843 - val_loss: 0.0445 - val_accuracy: 0.9833

In [41]:
oos_pred
x_test_array.shape[0]

6173

In [42]:
dummies_test.columns

Index(['DoS', 'Normal', 'Probe', 'R2L', 'U2R'], dtype='object')

In [43]:
test_y.value_counts()

Normal    2629
DoS       2263
Probe      782
R2L        486
U2R         13
Name: Class, dtype: int64

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
cm = confusion_matrix(y_eval, pred, labels=[0,1,2,3,4])

In [46]:
print(cm)

[[2258    5    0    0    0]
 [   6 2571    5   46    1]
 [   2    6  774    0    0]
 [   0   29    0  456    1]
 [   0    4    0    2    7]]


In [47]:
tp = 0
for i in range(5):
  for j in range(5):
    if i == j:
        tp = tp + cm[i][j]
tp_rate = (tp/24752)*100

print("True Positive Rate: " + str(tp_rate))

True Positive Rate: 24.5071105365223


In [48]:
dr = (cm[0][0] + cm[2][2] + + cm[3][3] + + cm[4][4])/(8897+2347+617+19)
print("Detection Rate: " + str(dr))

Detection Rate: 0.29419191919191917


In [1]:
model.save('BiLSTM-CNN.h5')
os.listdir()

def load_ids_model():
    ids_model = tf.keras.models.load_model(IDS_ModelPath)

NameError: name 'model' is not defined

In [74]:
print("Test loss:", result[0])
print("Test accuracy:", result[1])
result

Test loss: 0.01209273561835289
Test accuracy: 0.994853138923645


[0.01209273561835289, 0.994853138923645]

In [77]:
o_pred_label = tf.nn.softmax(loaded_model.predict(tf.expand_dims(new_train_df, axis=-1)), axis=1).numpy()


In [80]:
print(o_pred_label)


[[0.14920422 0.40177134 0.14920388 0.15061355 0.14920701]
 [0.14885905 0.40451995 0.14887287 0.1488892  0.14885892]
 [0.4046097  0.1488476  0.1488476  0.1488476  0.1488476 ]
 ...
 [0.40460467 0.14885059 0.14884828 0.1488482  0.1488482 ]
 [0.1488476  0.40460962 0.14884758 0.14884758 0.14884758]
 [0.1488476  0.1488476  0.4046097  0.1488476  0.1488476 ]]


In [82]:
o_pred_label = np.argmax(o_pred_label,axis=1)

In [86]:
import numpy as np
np.set_printoptions(threshold=np.inf)
print(o_pred_label)

[1 1 0 1 1 0 0 0 0 0 0 0 1 3 0 0 1 2 1 1 0 0 1 1 0 1 0 1 1 1 2 0 1 2 1 2 1
 0 1 0 0 0 1 1 1 0 0 1 3 0 1 0 0 1 1 1 2 0 0 1 1 0 2 1 0 1 2 0 0 0 1 0 1 1
 0 1 0 1 1 0 1 0 0 2 1 0 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 2 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 2 1
 3 0 0 1 0 0 0 1 1 0 2 0 1 1 1 1 0 2 1 1 1 1 0 1 0 0 0 0 1 2 2 0 1 1 1 1 0
 0 0 0 1 1 3 1 0 0 1 1 1 1 2 1 1 0 1 1 1 1 0 1 1 0 2 0 0 1 0 0 1 1 1 1 1 0
 3 0 2 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 1 2 0 0 0 1 0 2 0 1 2 1 1 2 2 0 1 1 2
 1 0 1 1 0 1 1 1 1 2 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 1 2 1 0 0 1 1 1
 1 2 1 0 1 0 0 3 1 2 1 1 2 1 0 1 0 0 1 0 2 2 1 0 1 0 0 0 1 0 1 1 1 1 2 2 1
 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 2 2 1 0 1 2 1 0 1 0 1 1 1 0 1 1 2 0 1 0 0
 1 1 1 0 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 0 0 0 0 2 0 2 1 1 0 1 1 2 1 1
 1 0 0 1 0 1 1 0 1 1 0 0 0 1 0 0 2 0 1 0 1 1 2 1 0 0 1 1 1 1 1 1 1 0 1 0 0
 1 1 0 0 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 1 0 2 1 0
 0 0 1 2 1 1 2 0 1 0 1 1 